In [1]:
# to generate list of PDB identifiers, run 'advanced search' on rcsb.org with the following query:
# Chemical ID(s): CA or MG or ZN or MN or FE or NI or NA or K or CL or CO and 
# Polymeric type is Free and Experimental Method is X-RAY and Resolution is 2.001 or less 
# and Chain Type: there is a Protein chain but not any DNA or RNA or Hybrid and 
# Representative Structures at 90% Sequence Identity

# list of PDB identifiers to consider:
train="final_PDB_ids_1.5A"
#/Users/shuminghao/Documents/ion classification/new dataset/final_PDB_ids_1.5A.txt
water=1                # take into account crystallographic waters: yes/no
d0=3.5                 # score distance cutoff (in Angstrom); default=3.5
dt=.1                  # bin width (in Angstrom); defalt=0.1
bin_cnt=int(20./dt)    # look at that many bins of width dt

In [2]:
import urllib
import time
from random import shuffle
from operator import itemgetter
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

path_PDB_local="/Users/shuminghao/Documents/ion classification/new dataset/PDB/%s.pdb"

# check if PDB files exists; otherwise, downlaod it to path_PDB_local
def fetch(PDB,het):
    got_it=0
    
    try:
        f = open(path_PDB_local%PDB,'r')
        got_it = 1      
    except:
        url = "https://files.rcsb.org/view/%s.pdb"%PDB
        f = requests.get(url, allow_redirects=True)
        o = open(path_PDB_local%PDB,'w')

    tmp=[]
    atoms=[]

    for line in f:      
        if line[:6]=="ENDMDL": break###      
        if line[:6]=="HETATM":
            at1=line[11:17].strip()
            at2=line[17:20].strip()
            at3=line[76:78].strip()

            if het == at3:
                x=float(line[30:38])
                y=float(line[38:46])
                z=float(line[46:54])

                tmp.append((x,y,z))
                
            if water:
                if line[17:20].strip()=="HOH" and line[76:78].strip()=="O":
                    x=float(line[30:38])
                    y=float(line[38:46])
                    z=float(line[46:54])
                    atoms.append(("HOH","HOH",(x,y,z)))

                
        elif line[:4]=="ATOM":
            atn=line[11:16].strip()
            res=line[17:20].strip()
            x=float(line[30:38])
            y=float(line[38:46])
            z=float(line[46:54])
            
            if not line[16] in [' ','A']: continue###
                
            atoms.append((atn,res,(x,y,z)))

        if not got_it: o.write(line)
    if not got_it: o.close()
    else:          f.close()

    return tmp,atoms
    # tmp is ion location
    # atoms are surrounding atoms

In [3]:
# dictionary of CHARMM36 atom types. treat all 'HIS' residues as "HSE" (neutral)

attype={\
        'ALA':{'N':'NH1','CA':'CT1','CB':'CT3','C':'C','O':'O'},\
        'ARG':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CT2','CD':'CT2','NE':'NC2','CZ':'C','NH1':'NC2','NH2':'NC2','C':'C','O':'O'},\
        'ASN':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CC','OD1':'O','ND2':'NH2','C':'C','O':'O'},\
        'ASP':{'N':'NH1','CA':'CT1','CB':'CT2A','CG':'CC','OD1':'OC','OD2':'OC','C':'C','O':'O'},\
        'CYS':{'N':'NH1','CA':'CT1','CB':'CT2','SG':'S','C':'C','O':'O'},\
        'GLN':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CT2','CD':'CC','OE1':'O','NE2':'NH2','C':'C','O':'O'},\
        'GLU':{'N':'NH1','CA':'CT1','CB':'CT2A','CG':'CT2','CD':'CC','OE1':'OC','OE2':'OC','C':'C','O':'O'},\
        'GLY':{'N':'NH1','CA':'CT2','C':'C','O':'O'},\
        'HIS':{'N':'NH1','CA':'CT1','CB':'CT2','ND1':'NR1','CG':'CPH1','CE1':'CPH2','NE2':'NR2','CD2':'CPH1','C':'C','O':'O'},\
        'ILE':{'N':'NH1','CA':'CT1','CB':'CT1','CG2':'CT3','CG1':'CT2','CD':'CT3','CD1':'CT3','C':'C','O':'O'},\
        'LEU':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CT1','CD1':'CT3','CD2':'CT3','C':'C','O':'O'},\
        'LYS':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CT2','CD':'CT2','CE':'CT2','NZ':'NH3','C':'C','O':'O'},\
        'MET':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CT2','SD':'S','CE':'CT3','C':'C','O':'O'},\
        'PHE':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CA','CD1':'CA','CE1':'CA','CZ':'CA','CD2':'CA','CE2':'CA','C':'C','O':'O'},\
        'PRO':{'N':'N','CD':'CP3','CA':'CP1','CB':'CP2','CG':'CP2','C':'C','O':'O'},\
        'SER':{'N':'NH1','CA':'CT1','CB':'CT2','OG':'OH1','C':'C','O':'O'},\
        'THR':{'N':'NH1','CA':'CT1','CB':'CT1','OG1':'OH1','CG2':'CT3','C':'C','O':'O'},\
        'TRP':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CY','CD1':'CA','NE1':'NY','CE2':'CPT','CD2':'CPT','CE3':'CAI','CZ3':'CA','CZ2':'CAI','CH2':'CA','C':'C','O':'O'},\
        'TYR':{'N':'NH1','CA':'CT1','CB':'CT2','CG':'CA','CD1':'CA','CE1':'CA','CZ':'CA','OH':'OH1','CD2':'CA','CE2':'CA','C':'C','O':'O'},\
        'VAL':{'N':'NH1','CA':'CT1','CB':'CT1','CG1':'CT3','CG2':'CT3','C':'C','O':'O'}\
       }


all_CHARMM=[]
for k in attype.keys():
    for k2 in attype[k].keys():
        all_CHARMM.append(attype[k][k2])
all_CHARMM=sorted(list(set(all_CHARMM)))


# # atom type assignment of ligand SH9 in MC4R PDB structure
# attype["SH9"]={}
# f=open("C:/Users/shuminghao/Documents/ion classification/new dataset/MC4att.txt",'r')
# for line in f:
#     a,b=line.strip().split()
#     attype["SH9"][a]=b
# f.close()

if water: attype["HOH"]={"HOH":"HOH"}
    
print (len(all_CHARMM))

27


In [4]:
def dist(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2)**.5

# given a PDB and an ion type, get the hist
def get_hist(PDB,het):
    hist_ = [[] for i in range(1000)]

    PDB_IDs = []
    #f = open(path_PDB_local%PDB,'r')
    f=open("/Users/shuminghao/Documents/ion classification/new dataset/PDB/%s.txt"%PDB,'r')
    #/Users/shuminghao/Documents/ion classification/new dataset/PDB/%s.pdb
    for line in f:
        for item in line.split(','):
            PDB_IDs.append(item.strip())
    f.close()

    for ID in sorted(list(set(PDB_IDs))):
        CA,atoms = fetch(ID,het)

        for ca in CA: # for each ion location
            for atom in atoms:

                atn,res,xyz=atom

                d = dist(ca,xyz)

                try:    
                    hist_[int(d/dt)].append(attype[res][atn])
                except: 
                    continue
    return hist_

In [5]:
all_atomtypes = []

for k in attype.keys():
    all_atomtypes += attype[k].keys()
    
all_atomtypes = list(set(all_atomtypes))

In [6]:
def get_total_counts(hist_):
    total_counts_ = {}

    for b in hist_[:bin_cnt]:
        for atom in b:
            if not atom in total_counts_.keys(): 
                total_counts_[atom] = 0
            total_counts_[atom] += 1

    return total_counts_
# e.g. {NH1: 12, CT1: 23...}

In [7]:
# smooth scoring function

def smooth(l):
    tmp=[]
    tmp.append(.5*(l[0]+l[1]))
    for i in range(len(l)-2): 
        tmp.append((l[i]+l[i+1]+l[i+2])/3.)
    tmp.append(.5*(l[-2]+l[-1]))
    return tmp


def get_score(total_counts_,hist_):
    score = {}
    for att in sorted(total_counts_.keys()): # e.g. att is NH1, CT1...

        score[att] = []

        rho = total_counts_[att]/(4./3*np.pi*(bin_cnt*dt)**3) 
        #?? total_counts_ca should be total_counts_?
        # rho is total_counts of the att / the sphere volumn, bin_cnt*dt is 20, why not just use 20?
        #print rho

        for i in range(bin_cnt):
            Vdt = 4./3*np.pi*((i+1)*dt)**3 - 4./3*np.pi*(i*dt)**3
            # volumn change in the new sphere

            s = hist_[i].count(att)/(rho*Vdt)

            score[att].append(s)

        # smooth scoring function here
        score[att] = smooth(score[att])
    return score

In [8]:
t0=time.time()

hist_ca = get_hist("%s"%train,"CA")

total_counts_ca = get_total_counts(hist_ca)

score_ca = get_score(total_counts_ca,hist_ca)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.47 minutes


In [9]:
t0=time.time()

hist_mg=get_hist("%s"%train,"MG")

total_counts_mg=get_total_counts(hist_mg)

score_mg=get_score(total_counts_mg,hist_mg)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.42 minutes


In [10]:
t0=time.time()

hist_zn=get_hist("%s"%train,"ZN")

total_counts_zn=get_total_counts(hist_zn)

score_zn=get_score(total_counts_zn,hist_zn)
print ("%.2f minutes"%((time.time()-t0)/60.))

0.41 minutes


In [11]:
t0=time.time()

hist_mn=get_hist("%s"%train,"MN")

total_counts_mn=get_total_counts(hist_mn)

score_mn=get_score(total_counts_mn,hist_mn)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.37 minutes


In [12]:
t0=time.time()

hist_fe=get_hist("%s"%train,"FE")

total_counts_fe=get_total_counts(hist_fe)

score_fe=get_score(total_counts_fe,hist_fe)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.36 minutes


In [13]:
t0=time.time()

hist_ni=get_hist("%s"%train,"NI")

total_counts_ni=get_total_counts(hist_ni)

score_ni=get_score(total_counts_ni,hist_ni)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.34 minutes


In [14]:
t0=time.time()

hist_co=get_hist("%s"%train,"CO")

total_counts_co=get_total_counts(hist_co)

score_co=get_score(total_counts_co,hist_co)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.34 minutes


In [15]:
t0=time.time()

hist_k=get_hist("%s"%train,"K")

total_counts_k=get_total_counts(hist_k)

score_k=get_score(total_counts_k,hist_k)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.35 minutes


In [16]:
t0=time.time()

hist_na=get_hist("%s"%train,"NA")

total_counts_na=get_total_counts(hist_na)

score_na=get_score(total_counts_na,hist_na)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.42 minutes


In [17]:
t0=time.time()

hist_cl=get_hist("%s"%train,"CL")

total_counts_cl=get_total_counts(hist_cl)

score_cl=get_score(total_counts_cl,hist_cl)

print ("%.2f minutes"%((time.time()-t0)/60.))

0.49 minutes


In [18]:
def score_it(het,score_,PDB_IDs_):
    score_dist_=[]
    for ID in PDB_IDs_:
        hets,atoms=fetch(ID,het)
        for het_ in hets:
            s0=0.
            for atom in atoms:
                atn,res,xyz=atom
                d=dist(het_,xyz)
                if d<d0:
                    try:    
                        s0-= np.log ( score_[attype[res][atn]][int(d/dt)] + 10.**-6 )
                    except: continue
            if abs(s0)<float('Inf'): score_dist_.append(-s0)
    return score_dist_


def score_it_full_result(het,score_,PDB_IDs_):
    score_dist_=[]
    for ID in PDB_IDs_:
        hets,atoms=fetch(ID,het)
        for het_ in hets:
            s0=0.
            for atom in atoms:
                atn,res,xyz=atom
                d=dist(het_,xyz)
                if d<d0:
                    try:    
                        s0-= np.log ( score_[attype[res][atn]][int(d/dt)] + 10.**-6 )
                    except: continue
            if abs(s0)<float('Inf'): score_dist_.append((ID,het,het_,-s0))
    return score_dist_

In [19]:
PDB_IDs=[]

f=open("/Users/shuminghao/Documents/ion classification/new dataset/%s.txt"%train,'r')
for line in f:
    for item in line.split(','):
        PDB_IDs.append(item.strip())
f.close()
PDB_IDs=sorted(list(set(PDB_IDs)))

In [20]:
if 0:
    t0=time.time()

    CA_scores=score_it('CA',score_ca,PDB_IDs)
    MG_scores=score_it('CA',score_mg,PDB_IDs)
    ZN_scores=score_it('CA',score_zn,PDB_IDs)

    MN_scores=score_it('CA',score_mn,PDB_IDs)
    FE_scores=score_it('CA',score_fe,PDB_IDs)
    NI_scores=score_it('CA',score_ni,PDB_IDs)
    CO_scores=score_it('CA',score_co,PDB_IDs)

    NA_scores=score_it('CA',score_na,PDB_IDs)
    K_scores =score_it('CA',score_k ,PDB_IDs)
    CL_scores=score_it('CA',score_cl,PDB_IDs)

    print ("%.2f minutes."%((time.time()-t0)/60.))

In [21]:
t0=time.time()
result=[]

hets={'CA':1,'MG':2,'ZN':3,'MN':4,'FE':5,'NI':6,'CO':7,'NA':8,'K':9,'CL':10}

for het in hets.keys():

    print (het)
    
    
    CA_scores=score_it( het , score_ca ,PDB_IDs)
    MG_scores=score_it( het , score_mg ,PDB_IDs)
    ZN_scores=score_it( het , score_zn ,PDB_IDs)

    MN_scores=score_it( het , score_mn ,PDB_IDs)
    FE_scores=score_it( het , score_fe ,PDB_IDs)
    NI_scores=score_it( het , score_ni ,PDB_IDs)
    CO_scores=score_it( het , score_co ,PDB_IDs)

    NA_scores=score_it( het , score_na ,PDB_IDs)
    K_scores =score_it( het , score_k  ,PDB_IDs)
    CL_scores=score_it( het , score_cl ,PDB_IDs)
    
    
    for i in range(len(CA_scores)):
        tmp=[hets[het],CA_scores[i],MG_scores[i],ZN_scores[i],MN_scores[i],FE_scores[i],NI_scores[i],CO_scores[i],NA_scores[i],K_scores[i],CL_scores[i]]
        result.append(tmp)

CA
MG
ZN
MN
FE
NI
CO
NA
K
CL


In [22]:
if water:
    o=open("/Users/shuminghao/Documents/ion classification/new dataset/descriptors-water.dat",'w')
else:    
    o=open("/Users/shuminghao/Documents/ion classification/new dataset/descriptors.dat",'w')
for r in result:
    o.write( "%d\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f\n"%tuple(r) )
o.close()
print ("%.2f minutes."%((time.time()-t0)/60.))

36.83 minutes.
